In [1]:
import pyodbc
import os
import pandas as pd
import numpy as np
import datetime
from datetime import datetime as dt
from dateutil.relativedelta import relativedelta
import getpass
import plotly
import plotly.express as px
import plotly.io as pio

import matplotlib.pyplot as plt
%matplotlib inline

start = dt.now()
today_date = dt.today().date()
tm_day_one = dt.today().date().replace(day=1)
tmly_day_one = tm_day_one - relativedelta(months=11)

replenish = tm_day_one - relativedelta(months=2)

print(start)
pd.set_option('display.max_columns', None)

2022-11-28 08:47:16.914659


In [2]:
# SQL python connection to our server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=sql-2-db;'
                      'Database=CBQ2;')
                    
cursor = conn.cursor()

# Amazon PO dataframes.

In [3]:
df_po = pd.read_csv(r'G:\SALES\Amazon\PURCHASE ORDERS\2022\rbh_po\po_analysis\PurchaseOrderItems.csv'
                    ,dtype={'External ID': 'object'})

In [4]:
# Summary
tc = sum(df_po['Total Cost'])
# print(f'There are {df_po.shape[0]} rows and the total cost is ${tc:,.0f}.')
print(tc)

905211.4800000013


In [5]:
# Aggregate the po doc so we have one ISBN per line
def po_clean(df):
    df = df.groupby(by=['ASIN','External ID']).agg(
        {'Accepted Quantity':'sum','Quantity Requested':'sum','Total Cost':'sum','Unit Cost':'mean'}).reset_index()
    df.rename(columns = {'External ID':'ISBN'},inplace = True)
    df['ISBN'] = df['ISBN'].str.zfill(13)
    df['ASIN'] = df['ASIN'].str.zfill(10)
    return df

In [6]:
# Clean PO table
df_po_clean = po_clean(df_po)
df_po_clean.head()

,ASIN,ISBN,Accepted Quantity,Quantity Requested,Total Cost,Unit Cost
0,0735330972,9780735330979,90,90,495.0,5.5
1,0735331006,9780735331006,72,72,468.0,6.5
2,073533112X,9780735331129,90,90,495.0,5.5
3,0735331294,9780735331297,0,64,0.0,8.5
4,073533305X,5711008911636,0,834,0.0,7.5


In [12]:
# Checking totals
print(round(sum(df_po_clean['Total Cost'])) == round(tc))
print(f'Total Cost: {tc:,.0f}')

True
Total Cost: 905,211


In [13]:
df_po_clean.isnull().sum()

ASIN                  0
ISBN                  0
Accepted Quantity     0
Quantity Requested    0
Total Cost            0
Unit Cost             0
dtype: int64

# Category info from the Product Catalog

In [14]:
df_cat = pd.read_csv(r'G:\SALES\Amazon\PURCHASE ORDERS\2022\rbh_po\amazon_product_catalog\Product Catalog_US.csv'
                    ,skiprows=1
                    ,usecols=['EAN','ASIN','ISBN-13','List Price','Binding','Product Group']
                    ,na_values=['UNKNOWN','—']
                    ,dtype={'EAN': 'object','ISBN-13':'object'})

In [15]:
def cat_clean(df):
    # remove where both EAN and ISBN-13 are both null
    filt1 = (df_cat['EAN'].isnull()) & (df_cat['ISBN-13'].isnull())
    df = df.loc[~filt1]
    
    # remove where list price = zero
    filt2 = df_cat['List Price'] == '$0.00'
    df = df.loc[~filt2]
    
    # drop list price column
    df.drop(['List Price'], axis=1,inplace=True)
    
    # Renaming the ean to ISBN
    df.rename(columns={"EAN": "ISBN"}, errors="raise",inplace = True)
    
    # Sometimes there is an ISBN-13 but no EAN, so i'll take the ISBN-13 if EAN is missing
    df['ISBN'] = np.where(df['ISBN'].isna(),df['ISBN-13'],df['ISBN'])
    
    # Removing the ISBN-13 since no longer needed
    df.drop(['ISBN-13'], axis=1,inplace=True)
    
    df['ISBN'] = df['ISBN'].str.zfill(13)
    df['ASIN'] = df['ASIN'].str.zfill(10)
    
    return df

In [16]:
# applying function
df_cat_clean = cat_clean(df_cat)

In [17]:
print(f'The clean cat table has {df_cat_clean.shape[0]} rows.')
print(f'The ASIN column as {df_cat_clean.ASIN.nunique()} unique values.')

The clean cat table has 26415 rows.
The ASIN column as 26415 unique values.


In [80]:
df_cat_clean.loc[df_cat_clean.duplicated(subset=['ISBN'])].sort_values(by='ISBN')

,ASIN,ISBN,Binding,Product Group


In [73]:
df_cat_clean.loc[df_cat_clean.ISBN =='9780811806763']

,ASIN,ISBN,Binding,Product Group
18391,B004VS7D9S,9780811806763,NaN,Digital_Ebook_Purchase
25184,0811806766,9780811806763,cards,Books


In [81]:
df_cat_clean = df_cat_clean.drop_duplicates(subset='ISBN', keep="first")

# Inventory Data from the Product Catalog

In [82]:
df_avail = pd.read_excel(r'G:\OPS\Inventory\Daily\Finance_Only\Inventory Detail.xlsx'
                        ,usecols=['ISBN','Available To Sell','Reprint Due Date','Reprint Quantity']
                        ,dtype={'ISBN': 'object'}
                        ,engine='openpyxl')

In [83]:
# Function to clean up the available table
# (1) removes null ISBNs, (2) makes ISBN's 13 digits long, (3) puts 0's in the empty "Reprint Quantity" bucket.

def avail_clean(df):
    df = df.loc[~df.ISBN.isnull()].reset_index(drop=True)
    df['ISBN'] = df['ISBN'].astype(str).apply(lambda x: x.zfill(13))
    df['Reprint Quantity'] = df['Reprint Quantity'].fillna(0) 
    df['ISBN'] = df['ISBN'].str.zfill(13)
    return df

In [84]:
# Clean up df_avail table from the Inventory Detail table
df_avail_clean = avail_clean(df_avail)

df_avail_clean.head()

,ISBN,Reprint Due Date,Reprint Quantity,Available To Sell
0,0030955767857,NaT,5004.0,2717.0
1,0030955767871,NaT,5004.0,4687.0
2,0736313543308,NaT,5004.0,2744.0
3,0736313543377,NaT,3024.0,2679.0
4,0736313543384,NaT,3024.0,1616.0


In [85]:
# There are no duplicate ISBN's in table
print(df_avail_clean.shape[0] == df_avail_clean.ISBN.nunique())
print(df_avail_clean.shape[0])

True
35616


# YPTICOD data

In [86]:
df_asin = pd.read_excel(r'\\sierra\groups\READTHIS\FINANCE\SALES\22sales\Oracle YPTICOD.xlsx'
                        ,usecols=['ISBN','ISBN10']
                        ,dtype={'ISBN': 'object','ISBN10':'object'}
                        ,engine='openpyxl')

In [87]:
def clean_ypticod(df):
    df['ISBN'] = df.loc[:,'ISBN'].astype(str).str.zfill(13)
    df['ISBN10'] = df.loc[:,'ISBN10'].astype(str).str.zfill(10)
    filt = df['ISBN10'].isna()
    df = df.loc[~filt]
    df.rename(columns = {'ISBN10':'ASIN'}, inplace = True)
    return df

In [88]:
# Applying the function
df_asin_clean = clean_ypticod(df_asin)

# Item Table Information

In [89]:
query_item = f'''
SELECT
    i.PUBLISHER_CODE Publisher
    ,i.PRODUCT_TYPE pt
    ,i.REPORTING_CATEGORY cat
    ,case
        when i.PUBLISHING_GROUP in('BAR-ENT','BAR-ART','BAR-FWN','BAR-LIF','BAR-CHL') then 'BAR'
        else i.PUBLISHING_GROUP
    end pgrp
    ,i.ITEM_TITLE ISBN
    ,i.SHORT_TITLE title
    ,i.PRICE_AMOUNT price
    ,convert(char,coalesce(convert(varchar,i.AMORTIZATION_DATE,101),shdt.shipdate),101) pub
    ,case
        when i.AMORTIZATION_DATE is not null then year(i.AMORTIZATION_DATE)
        when substring(i.season,1,4) <> 'No S' then substring(i.season,1,4)
        else year(getdate())
    end [year]

FROM ebs.Item i
    left join (SELECT [ISBN],[SHIPDATE] FROM [CBQ2].[pm].[ItemInfo] ii WHERE ii.IMPRESSION = 1 and ii.SHIPDATE is not null) shdt on shdt.ISBN = i.ISBN 
'''

In [90]:
df_item = pd.read_sql_query(query_item,conn)

In [91]:
df_item['pub'] = df_item['pub'].astype('datetime64[ns]')

df_item.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58937 entries, 0 to 58936
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Publisher  58937 non-null  object        
 1   pt         58937 non-null  object        
 2   cat        58937 non-null  object        
 3   pgrp       58937 non-null  object        
 4   ISBN       58937 non-null  object        
 5   title      58937 non-null  object        
 6   price      43346 non-null  float64       
 7   pub        39149 non-null  datetime64[ns]
 8   year       58937 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(1), object(6)
memory usage: 4.0+ MB


# Merge PO and Category Dataframes

In [92]:
def merge_cat_po(df_po,df_cat):
    df = pd.merge(df_po,df_cat,how = 'left',on = 'ISBN').fillna(0)
    return df

In [93]:
current_tc = round(sum(df_po['Total Cost']))
print(f'{current_tc:,.0f}')

905,211


In [94]:
# Combining the po dataframe and the category dataframe
df_cat_po = merge_cat_po(df_po_clean,df_cat_clean)

In [95]:
df_cat_po.info()

df_cat_po.rename(columns={'ASIN_x':'ASIN'},inplace=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2114 entries, 0 to 2113
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ASIN_x              2114 non-null   object 
 1   ISBN                2114 non-null   object 
 2   Accepted Quantity   2114 non-null   int64  
 3   Quantity Requested  2114 non-null   int64  
 4   Total Cost          2114 non-null   float64
 5   Unit Cost           2114 non-null   float64
 6   ASIN_y              2114 non-null   object 
 7   Binding             2114 non-null   object 
 8   Product Group       2114 non-null   object 
dtypes: float64(2), int64(2), object(5)
memory usage: 165.2+ KB


In [96]:
sum(df_cat_po['Total Cost'])

905211.4800000007

In [97]:
df_cat_po['ASIN_y'].nunique()

2079

# Merge Tables

In [98]:
# combines both the df_po and df_item table and provides just the columns we want

def merge_clean_tables(df_po,df_item,df_avail):
    df_po['ISBN'] = df_po['ISBN'].str.zfill(13)
    df_avail['ISBN'] = df_avail['ISBN'].str.zfill(13)
    df_item['ISBN'] = df_item['ISBN'].str.zfill(13)
    df_po = df_po.loc[:,['ASIN','ISBN','Binding','Product Group','Accepted Quantity','Quantity Requested','Total Cost','Unit Cost']]
    df_item = df_item.loc[:,['ISBN','Publisher','pgrp','title','pub','price']]    
    #merge po and item tables
    df = pd.merge(df_po,df_item,how = 'left',left_on = 'ISBN',right_on = 'ISBN').fillna(0)
    # add variance column
    df['Delta'] = df['Accepted Quantity']-df['Quantity Requested']
    #merge df_avail
    df = pd.merge(df,df_avail,how='left',on='ISBN')
    #Creating the "sales lost column"
    df['Lost Sales'] = (df['Quantity Requested']*df['Unit Cost'])-df['Total Cost']
    #specify the order and columns we want in the final product
    df = df.loc[:,['ASIN','ISBN','title','Publisher','pgrp','pub'
                   ,'Product Group','Available To Sell','Reprint Due Date'
                   ,'Reprint Quantity','Quantity Requested','Accepted Quantity'
                   ,'Delta','Total Cost','Lost Sales']]
    return df

In [99]:
df_combined = merge_clean_tables(df_cat_po,df_item,df_avail_clean)
print(df_combined.shape[0])

df_combined.shape

2114


(2114, 15)

# Basic Summaries

In [100]:
df_combo_grp = df_combined.groupby(by=['Publisher']).agg({'Total Cost':'sum'})
df_combo_grp

,Total Cost
Publisher,
0,48.83
AMMO Books,50.54
Chronicle,446753.11
Creative Company,316.94
Galison,315781.44
Hardie Grant Publishing,51263.32
Laurence King,43322.98
Levine Querido,3436.20
Princeton,29140.00


# Clean Merge

In [101]:
print(f"The PO table has {df_po_clean.shape[0]} rows.")
print(f"The combined table has {df_combined.shape[0]} rows.")

total_cost = sum(df_combined['Total Cost'])
print(f'The total "Total Cost" is {total_cost:,.0f}.')

The PO table has 2114 rows.
The combined table has 2114 rows.
The total "Total Cost" is 905,211.


In [102]:
df_combined[(df_combined['Publisher'] == 0)]

,ASIN,ISBN,title,Publisher,pgrp,pub,Product Group,Available To Sell,Reprint Due Date,Reprint Quantity,Quantity Requested,Accepted Quantity,Delta,Total Cost,Lost Sales
4,073533305X,5711008911636,0,0,0,0,0,NaN,NaT,NaN,834,0,-834,0.00,6255.0
8,073533417X,0081177766630,0,0,0,0,Toys,NaN,NaT,NaN,162,0,-162,0.00,1215.0
56,0735351740,0492019587946,0,0,0,0,Toys,NaN,NaT,NaN,100,0,-100,0.00,644.0
744,141970883X,9781419708831,0,0,0,0,0,NaN,NaT,NaN,1,1,0,5.85,0.0
1317,158479853X,9781584798538,0,0,0,0,0,NaN,NaT,NaN,1,1,0,12.38,0.0
1318,1585670219,9781585670215,0,0,0,0,0,NaN,NaT,NaN,4,4,0,30.60,0.0


In [103]:
# Removing the non-Chronicle titles from the list
df_combined_clean = df_combined[~(df_combined['Publisher'] == 0)]

In [104]:
sum(df_combined_clean['Total Cost'])

905162.6500000007

In [105]:
# I'm removing the ISBN's that don't actually belong to any CB or DP. See list below
df_combined_clean = df_combined[~(df_combined['Publisher'] == 0)]

# These orders are NOT for any of our titles.
print(df_combined[(df_combined['Publisher'] == 0)].shape[0])
df_combined[(df_combined['Publisher'] == 0)]

6


,ASIN,ISBN,title,Publisher,pgrp,pub,Product Group,Available To Sell,Reprint Due Date,Reprint Quantity,Quantity Requested,Accepted Quantity,Delta,Total Cost,Lost Sales
4,073533305X,5711008911636,0,0,0,0,0,NaN,NaT,NaN,834,0,-834,0.00,6255.0
8,073533417X,0081177766630,0,0,0,0,Toys,NaN,NaT,NaN,162,0,-162,0.00,1215.0
56,0735351740,0492019587946,0,0,0,0,Toys,NaN,NaT,NaN,100,0,-100,0.00,644.0
744,141970883X,9781419708831,0,0,0,0,0,NaN,NaT,NaN,1,1,0,5.85,0.0
1317,158479853X,9781584798538,0,0,0,0,0,NaN,NaT,NaN,1,1,0,12.38,0.0
1318,1585670219,9781585670215,0,0,0,0,0,NaN,NaT,NaN,4,4,0,30.60,0.0


In [106]:
filt = (df_combined_clean['Publisher'] == 0)

df_pub_summary= df_combined_clean[~filt].groupby(by=['Publisher'])[['Quantity Requested','Accepted Quantity','Delta','Total Cost','Lost Sales']].sum().reset_index()
df_pub_summary.sort_values(by=['Total Cost'],ascending=False,inplace=True)
df_pub_summary

,Publisher,Quantity Requested,Accepted Quantity,Delta,Total Cost,Lost Sales
1,Chronicle,61325,49871,-11454,446753.11,74998.43
3,Galison,57027,43252,-13775,315781.44,104992.00
4,Hardie Grant Publishing,4583,3888,-695,51263.32,8383.96
5,Laurence King,7759,4411,-3348,43322.98,32467.48
7,Princeton,2097,1843,-254,29140.00,3101.72
8,Tourbillon,1558,1488,-70,15098.12,598.92
6,Levine Querido,427,400,-27,3436.20,232.76
2,Creative Company,138,67,-71,316.94,571.78
0,AMMO Books,4,4,0,50.54,0.00


In [107]:
df_top_ordered_cb = df_combined_clean[df_combined_clean['Publisher']=='Chronicle'].sort_values(by=['Quantity Requested'],ascending=False).head(20).reset_index(drop=True)
df_top_ordered_cb.head(5)

,ASIN,ISBN,title,Publisher,pgrp,pub,Product Group,Available To Sell,Reprint Due Date,Reprint Quantity,Quantity Requested,Accepted Quantity,Delta,Total Cost,Lost Sales
0,1452174792,9781452174792,Canvas One Line A Day,Chronicle,ART,2019-01-01 00:00:00,Books,55303.0,NaT,50000.0,2942,2880,-62,22464.0,4.836000e+02
1,B07FGPB61X,5055923751886,Game Avocado Smash,Chronicle,RID,2021-01-01 00:00:00,Toys,5.0,2022-02-21,7524.0,2565,135,-2430,745.2,1.341360e+04
2,0811868958,9780811868952,Just Between Us: Mother & Daughter,Chronicle,CHL,2010-03-01 00:00:00,Books,26499.0,NaT,25000.0,2528,2528,0,19718.4,-3.637979e-12
3,1452180733,9781452180731,100 Cookies,Chronicle,FWN,2020-08-01 00:00:00,Books,47187.0,NaT,40000.0,2265,2254,-11,28513.1,1.391500e+02
4,0811870197,9780811870191,One Line a Day,Chronicle,ART,2009-09-01 00:00:00,Books,43051.0,NaT,0.0,1844,1800,-44,14040.0,3.432000e+02


In [108]:
df_top_ordered_dp = df_combined_clean[~(df_combined_clean['Publisher']=='Chronicle')].sort_values(by=['Quantity Requested'],ascending=False).head(20).reset_index(drop=True)
df_top_ordered_dp.head(5)

,ASIN,ISBN,title,Publisher,pgrp,pub,Product Group,Available To Sell,Reprint Due Date,Reprint Quantity,Quantity Requested,Accepted Quantity,Delta,Total Cost,Lost Sales
0,0735375283,9780735375284,Puz 1000 Autumn In the Neighborhood,Galison,GAL,2022-06-01 00:00:00,Toys,-741.0,NaT,5500.0,3330,0,-3330,0.0,29970.0
1,0735362149,9780735362147,Puz 500 Foil Winter Lights Foil,Galison,GAL,2020-08-01 00:00:00,Toys,-3262.0,2023-01-27,3000.0,2840,0,-2840,0.0,21300.0
2,0735366721,9780735366725,Puz 500 Foil Merry Moonlight Skaters,Galison,GAL,2021-06-01 00:00:00,Toys,10640.0,NaT,0.0,2360,2360,0,17700.0,0.0
3,1786277476,9781786277473,Poop Bingo,Laurence King,LAU-GIFT,2020-08-01 00:00:00,Toys,-2487.0,2022-11-29,12000.0,1281,0,-1281,0.0,11785.2
4,0735371180,9780735371187,Puz 1000 SQ Joy Laforme Christmas Square,Galison,GAL,2022-08-01 00:00:00,Toys,-213.0,NaT,0.0,1180,0,-1180,0.0,10030.0


In [109]:
df_top_accepted_cb = df_combined_clean[df_combined_clean['Publisher']=='Chronicle'].sort_values(by=['Accepted Quantity'],ascending=False).head(20).reset_index(drop=True)
df_top_accepted_cb.head(5)

,ASIN,ISBN,title,Publisher,pgrp,pub,Product Group,Available To Sell,Reprint Due Date,Reprint Quantity,Quantity Requested,Accepted Quantity,Delta,Total Cost,Lost Sales
0,1452174792,9781452174792,Canvas One Line A Day,Chronicle,ART,2019-01-01 00:00:00,Books,55303.0,NaT,50000.0,2942,2880,-62,22464.0,4.836000e+02
1,0811868958,9780811868952,Just Between Us: Mother & Daughter,Chronicle,CHL,2010-03-01 00:00:00,Books,26499.0,NaT,25000.0,2528,2528,0,19718.4,-3.637979e-12
2,1452180733,9781452180731,100 Cookies,Chronicle,FWN,2020-08-01 00:00:00,Books,47187.0,NaT,40000.0,2265,2254,-11,28513.1,1.391500e+02
3,0811870197,9780811870191,One Line a Day,Chronicle,ART,2009-09-01 00:00:00,Books,43051.0,NaT,0.0,1844,1800,-44,14040.0,3.432000e+02
4,0811870413,9780811870412,Tartine Bread,Chronicle,FWN,2010-09-01 00:00:00,Books,20248.0,NaT,0.0,1770,1770,0,32568.0,-3.637979e-12


In [110]:
df_top_accepted_dp = df_combined_clean[~(df_combined_clean['Publisher']=='Chronicle')].sort_values(by=['Accepted Quantity'],ascending=False).head(20).reset_index(drop=True)
df_top_accepted_dp.head(5)

,ASIN,ISBN,title,Publisher,pgrp,pub,Product Group,Available To Sell,Reprint Due Date,Reprint Quantity,Quantity Requested,Accepted Quantity,Delta,Total Cost,Lost Sales
0,0735366721,9780735366725,Puz 500 Foil Merry Moonlight Skaters,Galison,GAL,2021-06-01 00:00:00,Toys,10640.0,NaT,0.0,2360,2360,0,17700.0,0.0
1,0735360731,9780735360730,Puz 100 Double Side Cats & Dogs,Galison,MUD,2019-06-01 00:00:00,Toys,4743.0,2022-04-14,3000.0,910,910,0,7735.0,0.0
2,0735351937,9780735351936,Puz 70 Geography Map of USA,Galison,MUD,2017-06-01 00:00:00,Toys,13881.0,NaT,0.0,750,750,0,6375.0,0.0
3,0735362017,9780735362017,Puz 500 Winter Dogs,Galison,GAL,2020-08-01 00:00:00,Toys,8773.0,NaT,11500.0,720,720,0,5040.0,0.0
4,0735349088,9780735349087,Puz 1000 Family Cool Cats A-Z,Galison,MUD,2016-11-01 00:00:00,Toys,27433.0,NaT,0.0,700,700,0,5950.0,0.0


In [111]:
# dataframe of titles that were requested but not accepted
df_lost_sales = df_combined_clean.sort_values(by=['Lost Sales'],ascending=False).head(20).reset_index(drop=True)

In [112]:
# Creating tables to summarize sales by product group

filt = (df_combined_clean['Publisher']=='Chronicle')

cb_product_group = df_combined_clean[filt].groupby(['Publisher','Product Group'])['Total Cost'].sum().reset_index()
dp_product_group = df_combined_clean[~filt].groupby(['Publisher','Product Group'])['Total Cost'].sum().reset_index()

In [113]:
total_cost_combined_clean = df_combined_clean['Total Cost'].sum()
cb_total_cost_combined_clean = cb_product_group['Total Cost'].sum()
dp_total_cost_combined_clean = dp_product_group['Total Cost'].sum()


print(f'Total should be {total_cost_combined_clean:,.0f}.')
print()
print(f'CB total should be {cb_total_cost_combined_clean:,.0f}.')
print(f'DP total should be {dp_total_cost_combined_clean:,.0f}.')

print(f'CB + DP total should be {cb_total_cost_combined_clean + dp_total_cost_combined_clean:,.0f}.')

Total should be 905,163.

CB total should be 446,753.
DP total should be 458,410.
CB + DP total should be 905,163.


In [114]:
print(cb_product_group.head())

df_combined_clean.loc[df_combined_clean['Product Group'] == 'Digital_Ebook_Purchase']

   Publisher Product Group  Total Cost
0  Chronicle             0     1262.24
1  Chronicle          Baby     7376.56
2  Chronicle         Books   361008.15
3  Chronicle          Home      587.20
4  Chronicle       Kitchen      552.00


,ASIN,ISBN,title,Publisher,pgrp,pub,Product Group,Available To Sell,Reprint Due Date,Reprint Quantity,Quantity Requested,Accepted Quantity,Delta,Total Cost,Lost Sales


In [117]:
# Used to provide a table for the tables that were rejected
reject_rows = df_combined.loc[df_combined['Publisher'] == 0]

In [118]:
path = r'G:\SALES\Amazon\PURCHASE ORDERS\2022\rbh_po\po_analysis\amazon_order_py_dump.xlsx'
writer = pd.ExcelWriter(path, engine='xlsxwriter')

df_pub_summary.to_excel(writer,sheet_name='pub_summary',index = False)
df_top_ordered_cb.to_excel(writer,sheet_name='ordered_summary_cb',index = False)
df_top_ordered_dp.to_excel(writer,sheet_name='ordered_summary_dp',index = False)
df_top_accepted_cb.to_excel(writer,sheet_name='accepted_summary_cb',index=False)
df_top_accepted_dp.to_excel(writer,sheet_name='accepted_summary_dp',index=False)
df_lost_sales.to_excel(writer,sheet_name='lost_sales_summary',index=False)
cb_product_group.to_excel(writer,sheet_name='cb_group_summary',index=False)
dp_product_group.to_excel(writer,sheet_name='dp_group_summary',index=False)
reject_rows.to_excel(writer,sheet_name='reject_summary',index=False)

writer.save()

# Creating df's for DPs

In [119]:
df_combined.dtypes

ASIN                          object
ISBN                          object
title                         object
Publisher                     object
pgrp                          object
pub                           object
Product Group                 object
Available To Sell            float64
Reprint Due Date      datetime64[ns]
Reprint Quantity             float64
Quantity Requested             int64
Accepted Quantity              int64
Delta                          int64
Total Cost                   float64
Lost Sales                   float64
dtype: object

In [120]:
df_combined.pub.replace(0,pd.NaT,inplace = True)

In [121]:
# df_temp.groupby(['ASIN','ISBN','title','pub','Reprint Due Date','Reprint Quantity','pgrp'])['Quantity Requested'].count()
df_combined_dp = df_combined.copy()
df_combined_dp.dtypes

ASIN                          object
ISBN                          object
title                         object
Publisher                     object
pgrp                          object
pub                   datetime64[ns]
Product Group                 object
Available To Sell            float64
Reprint Due Date      datetime64[ns]
Reprint Quantity             float64
Quantity Requested             int64
Accepted Quantity              int64
Delta                          int64
Total Cost                   float64
Lost Sales                   float64
dtype: object

In [122]:
df_combined_dp['pub'] = df_combined_dp.loc[:,'pub'].dt.strftime('%Y-%m-%d')
df_combined_dp['pub'].fillna("-",inplace=True)

df_combined_dp['Reprint Due Date'] = df_combined_dp.loc[:,'Reprint Due Date'].dt.strftime('%Y-%m-%d')
df_combined_dp['Reprint Due Date'].fillna("-",inplace=True)
df_combined_dp['Available To Sell'].fillna(0,inplace=True)
df_combined_dp['Reprint Quantity'].fillna(0,inplace=True)

df_combined_dp.isnull().sum()

ASIN                  0
ISBN                  0
title                 0
Publisher             0
pgrp                  0
pub                   0
Product Group         0
Available To Sell     0
Reprint Due Date      0
Reprint Quantity      0
Quantity Requested    0
Accepted Quantity     0
Delta                 0
Total Cost            0
Lost Sales            0
dtype: int64

In [123]:
# Function to create dp list without 0 and "Chronicle"
def create_dp_list(col):
    dp_list = list(col)
    if 0 in dp_list:
        dp_list.remove(0)
    if 'Chronicle' in dp_list:
        dp_list.remove('Chronicle')
    return dp_list

In [124]:
# Create DP List.
dp_list = create_dp_list(df_combined_dp.Publisher.unique())

dp_list

['Galison',
 'Laurence King',
 'Creative Company',
 'Princeton',
 'AMMO Books',
 'Levine Querido',
 'Hardie Grant Publishing',
 'Tourbillon']

In [125]:
# filter the table to single dp
def filter_dp(df,dp):
    df_new = df.loc[(df['Publisher']==dp)]
    return df_new

In [126]:
# creates a pub by pgrp view
def dp_by_pgrp(df):
    df_dp = df.groupby(
        ['pgrp']).agg(
        Sum_of_Total_Cost = pd.NamedAgg(column='Total Cost',aggfunc=sum))
    return df_dp

In [127]:
# top twenty
def dp_top20(df,col):
    df_dp = df.groupby(
        ['ASIN','ISBN','title','pub','Reprint Due Date','Reprint Quantity','pgrp']).agg(
        Quantity_Requested = pd.NamedAgg(column='Quantity Requested',aggfunc=sum)
        ,Quantity_Accepted = pd.NamedAgg(column='Accepted Quantity',aggfunc=sum)
        ,Total_Cost = pd.NamedAgg(column='Total Cost',aggfunc=sum))
    df_dp = df_dp.sort_values(by=col,ascending = False).head(20)
    
    return df_dp

In [128]:
# Creates dictionary and runs all three dataframes for each publisher.
dp_dict = {}

for dp in dp_list:
    df_temp = filter_dp(df_combined_dp,dp)
    dp_dict[dp] = list()
    dp_dict[dp].append(dp_by_pgrp(df_temp).reset_index(drop=False))
    dp_dict[dp].append(dp_top20(df_temp,col = 'Quantity_Requested').reset_index(drop=False))
    dp_dict[dp].append(dp_top20(df_temp,col = 'Quantity_Accepted').reset_index(drop=False))

In [129]:
path = r'G:\SALES\Amazon\PURCHASE ORDERS\2022\rbh_po\po_analysis\amazon_order_dp_dump.xlsx'
writer = pd.ExcelWriter(path, engine='xlsxwriter')

for dp in list(dp_dict.keys()):
    dp_dict[dp][0].to_excel(writer,sheet_name=f'{dp}_PGRP',index = False)
    dp_dict[dp][1].to_excel(writer,sheet_name=f'{dp}_QtyRqt',index = False)
    dp_dict[dp][2].to_excel(writer,sheet_name=f'{dp}_QtyAcpt',index = False)

writer.save()